In [ ]:
import pickle
import numpy as np
import pandas as pd
from PIL import Image
import albumentations as A
from IPython.display import SVG
import matplotlib.pyplot as plt
%matplotlib inline
import os, re, sys, random, shutil, cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras import applications, optimizers
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import model_to_dot, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, LearningRateScheduler
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, ZeroPadding2D, Dropout

In [ ]:
def augment(width, height):
    transform = A.Compose([
        A.RandomCrop(width=width, height=height, p=1.0),
        A.HorizontalFlip(p=1.0),
        A.VerticalFlip(p=1.0),
        A.Rotate(limit=[60, 300], p=1.0, interpolation=cv2.INTER_NEAREST),
        A.RandomBrightnessContrast(brightness_limit=[-0.2, 0.3], contrast_limit=0.2, p=1.0),
        A.OneOf([
            A.CLAHE (clip_limit=1.5, tile_grid_size=(8, 8), p=0.5),
            A.GridDistortion(p=0.5),
            A.OpticalDistortion(distort_limit=1, shift_limit=0.5, interpolation=cv2.INTER_NEAREST, p=0.5),
        ], p=1.0),
    ], p=1.0)

    return transform

In [ ]:
def visualize(image, mask, original_image=None, original_mask=None):
    fontsize = 16

    if original_image is None and original_mask is None:
        f, ax = plt.subplots(2, 1, figsize=(10, 10), squeeze=True)
        f.set_tight_layout(h_pad=5, w_pad=5)

        ax[0].imshow(image)
        ax[1].imshow(mask)
    else:
        f, ax = plt.subplots(2, 2, figsize=(16, 12), squeeze=True)
        plt.tight_layout(pad=0.2, w_pad=1.0, h_pad=0.01)

        ax[0, 0].imshow(original_image)
        ax[0, 0].set_title('Original Image', fontsize=fontsize)

        ax[1, 0].imshow(original_mask)
        ax[1, 0].set_title('Original Mask', fontsize=fontsize)

        ax[0, 1].imshow(image)
        ax[0, 1].set_title('Transformed Image', fontsize=fontsize)

        ax[1, 1].imshow(mask)
        ax[1, 1].set_title('Transformed Mask', fontsize=fontsize)

    plt.savefig('sample_augmented_image.png', facecolor= 'w', transparent= False, bbox_inches= 'tight', dpi= 100)

In [ ]:
#image = cv2.imread("../input/dubai-aerial-imagery-dataset/train_images/train/image_t8_007.jpg")
#image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#mask = cv2.imread("../input/dubai-aerial-imagery-dataset/train_masks/train/image_t8_007.png")
#mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

#transform = augment(1920, 1280)
#transformed = transform(image=image, mask=mask)
#transformed_image = transformed['image']
#transformed_mask = transformed['mask']

#cv2.imwrite('./image.png',cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB))
#cv2.imwrite('./mask.png',cv2.cvtColor(transformed_mask, cv2.COLOR_BGR2RGB))

#visualize(transformed_image, transformed_mask, image, mask)

In [ ]:
#!mkdir aug_images
#!mkdir aug_masks

In [ ]:
images_dir = '/content/drive/MyDrive/Colab Notebooks/datasets/Semantic segmentation dataset/train_images/'
masks_dir = '/content/drive/MyDrive/Colab Notebooks/datasets/Semantic segmentation dataset/train_masks/'

In [ ]:
file_names = np.sort(os.listdir(images_dir))
file_names = np.char.split(file_names, '_')
filenames = np.array([])
for i in range(len(file_names)):
    filenames = np.append(filenames, file_names[i][1])

In [ ]:
fin = []

# Iterate through the original list and add values to the new list if they are not already in it
for value in filenames:
    if value not in fin:
        fin.append(value)

# Now, unique_values contains the list without repeated values
print(fin)

['t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8']


### Augmenation

In [ ]:
import cv2
import os
from google.colab import drive  # Import the drive module

# Mount Google Drive to access and save files
drive.mount('/content/drive')

# Define the directories for images and masks in Google Drive
images_dir = '/content/drive/MyDrive/Colab Notebooks/datasets/Semantic segmentation dataset/train_images/'
masks_dir = '/content/drive/MyDrive/Colab Notebooks/datasets/Semantic segmentation dataset/train_masks/'

# Define the count of images you want to augment
count = 8  # Adjust this value as needed

# Define your augmentation transforms (you can modify this part based on your actual augmentations)
transform_1 = augment(512, 512)
transform_2 = augment(480, 480)
transform_3 = augment(512, 512)
transform_4 = augment(800, 800)
transform_5 = augment(1024, 1024)
transform_6 = augment(800, 800)
transform_7 = augment(1600, 1600)
transform_8 = augment(1920, 1280)


for i in range(1, 9):
 for j in range(1, 9):
    for file in filenames:
        tile = file

        # Construct the image and mask filenames based on your naming convention
        image_name = f'{images_dir}image_{tile}_00{j}.jpg'
        mask_name = f'{masks_dir}image_{tile}_00{j}.png'

        img = cv2.imread(image_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(mask_name)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

        if tile == 't1':
            transformed = transform_1(image=img, mask=mask)
        elif tile == 't2':
            transformed = transform_2(image=img, mask=mask)
        elif tile == 't3':
            transformed = transform_3(image=img, mask=mask)
        elif tile == 't4':
            transformed = transform_4(image=img, mask=mask)
        elif tile == 't5':
            transformed = transform_5(image=img, mask=mask)
        elif tile == 't6':
            transformed = transform_6(image=img, mask=mask)
        elif tile == 't7':
            transformed = transform_7(image=img, mask=mask)
        elif tile == 't8':
            transformed = transform_8(image=img, mask=mask)

        transformed_image = transformed['image']
        transformed_mask = transformed['mask']

        # Save the transformed images and masks in Google Drive
        output_image_path = f'/content/drive/MyDrive/Colab Notebooks/datasets/Aug/aug_image/aug_{file}_00{j}_{i}.jpg'
        output_mask_path = f'/content/drive/MyDrive/Colab Notebooks/datasets/Aug/aug_mask/aug_{file}_00{j}_{i}.png'

        # Ensure the output directories exist
        os.makedirs(os.path.dirname(output_image_path), exist_ok=True)
        os.makedirs(os.path.dirname(output_mask_path), exist_ok=True)

        cv2.imwrite(output_image_path, cv2.cvtColor(transformed_image, cv2.COLOR_RGB2BGR))
        cv2.imwrite(output_mask_path, cv2.cvtColor(transformed_mask, cv2.COLOR_RGB2BGR))
        # Explicitly flush and close the files


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
